In [1]:
import os
import pandas as pd
from unidecode import unidecode
import numpy as np
import re
import importlib
import global_settings
import hms_mh_cleansing_before_unification_function
import hms_mh_unification_after_cleansing_function


importlib.reload(global_settings)
importlib.reload(hms_mh_cleansing_before_unification_function)
importlib.reload(hms_mh_unification_after_cleansing_function)

<module 'hms_mh_unification_after_cleansing_function' from '/data/workdirectory/03. HMS unification/5. 20250320/MC/hms_mh_unification_after_cleansing_function.py'>

In [2]:
setts = global_settings.Settings()
cleansing_function = hms_mh_cleansing_before_unification_function.cleansing_function
unification_old_new_function = hms_mh_unification_after_cleansing_function.unification_old_new_function

province = pd.read_csv(setts.input_folder + "/province.csv",dtype=str)
district = pd.read_csv(setts.input_folder + "/district.csv",dtype=str)
convert_phonenumber = pd.read_csv(setts.input_folder + "/convert_phonenumber.csv",dtype=str)

Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input/tmp1
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input/tmp2
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/output


In [3]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

In [4]:
# case = 'case4'
case = 'match_vin_only'
output_path = setts.output_folder + '/cluster_' + case
create_folder_if_not_exists(output_path)
print(output_path)

df_combi = pd.read_csv(setts.tmp2_folder + '/hms_mh_%s.csv' % case,dtype=str).fillna('')
df_combi

/data/workdirectory/03. HMS unification/5. 20250320/MC/output/cluster_match_vin_only


,unification_id,customer_code,vin,customer_name,customer_address,mobile_phone,district,city,cluster
0,0e5d2910531d6bab3c73c9cacfbcbf85d4653cc7c3070e...,MH_4152397,000002883K5133876,NGO TUAN DUNG,LONG BIÊN HÀ NỘI,0904382002,longbien,hanoi,0
1,94b327f43e3666ef5f167105bddf659f4f3c1cd17f6dc8...,MH_3401434,000004322AY237320,HUỲNH VĂN MẪN,42 6 HOÀNG BẬT ĐẠT P 15 TÂY HOÀ HỒ CHÍ MINH,0399911084,other,hochiminh,0
2,953378e6739b8afe1055171648fc68805c5e805e3e48db...,MH_225318,00000C100M0192976,BẠCH QUỐC VIỆT,SỐ 21 LK25 KĐT 5A PHƯỜNG 4 SÓC TRĂNG,0939952777,other,soctrang,0
3,1273a44fd422bcef0ec306ca6ca89bbf0430c480d94979...,MH_425091,00000KC26E1043408,TRẦN VĂN THÌN,TRƯỜNG KHÁNH LỘC NINH SÓC TRĂNG,0868089841,other,soctrang,0
4,953378e6739b8afe1055171648fc68805c5e805e3e48db...,MH_225318,1010KY137909EXCIT,BẠCH QUỐC VIỆT,SỐ 21 LK25 KĐT 5A PHƯỜNG 4 SÓC TRĂNG,0939952777,other,soctrang,0
...,...,...,...,...,...,...,...,...,...
161854,3a022c5873a7cccaf2f7331894b24eacdfb101b9826822...,MH_1068484,WAVE5266DY0220402,NGUYỄN THANH BÌNH,THÔN 2 XÃ EAĐAR HUYỆN EAKAR TỈNH ĐĂKLAK EA KAR...,0365834613,eakar,daklak,0
161855,a2f4bc0f623d3ab58889c9e2f497c218c104440569b40c...,MH_469058,Y053745JC76E10769,LÊ THỊ ĐẬM,HỒ HỌC LÃM KHÁNH VĨNH HỒ CHÍ MINH,0907198818,other,hochiminh,0
161856,1974d0555494f000377fb8d11d52a8fc9734af54a82058...,MH_9012292,ZDCKF10B0AF059950,TRẦN THANH HẬN,PHƯỚC LÝ CẦN GIUỘC LONG AN PHAN THIẾT LONG AN,0982957990,other,longan,0
161857,876ccf6afc414404da683fa51ddc13e621650d6b12faa9...,MH_2622484,ZDCKF10BOAF056343,NGUYEN VAN DO,HỒ CHÍ MINH,0933383338,other,hochiminh,0


In [5]:
cluster_list = df_combi['cluster'].astype(int).drop_duplicates().sort_values().tolist()
print(cluster_list)

[0]


In [6]:
def generate_key(vin, name, address, phone, district, city, cut=False):
    if cut:
        name = hms_mh_cleansing_before_unification_function.clean_name_old_data(name)
        address = hms_mh_cleansing_before_unification_function.clean_address_old_data(address, district, city)
    
    return (vin+name+address+phone + district + city).lower().replace(' ', '')

df_combi['key'] = df_combi.apply(lambda x: generate_key(x['vin'], 
                                                          x['customer_name'], 
                                                          x['customer_address'], 
                                                          x['mobile_phone'], 
                                                          x['district'], 
                                                          x['city'], cut=True), axis=1)

In [7]:
df_old = df_combi[df_combi['unification_id'].str.len() != 0]
df_new = df_combi[df_combi['unification_id'].str.len() == 0]

print(len(df_old))
print(len(df_new))

92209
69650


In [8]:
# Go through each cluster
for cluster in cluster_list:
    print('=' * 50)
    print(f'Unifying case "{case}" cluster "{cluster}"')
    
    df_combi_cluster = df_combi[df_combi['cluster'] == str(cluster)]
    output_name = output_path + '/hms_mh_old_new_unified_cluster_%s.csv' % str(cluster)
    
    # Extract new and old data of cluster
    df_old_cluster = df_old[df_old['cluster'] == str(cluster)]
    df_new_cluster = df_new[df_new['cluster'] == str(cluster)]

    
    # Check if there are new data in cluster
    if len(df_new_cluster) == 0:
        print(f'No new data in cluster "{cluster}" of case "{case}", skipping')
        continue
        
    # Clean old & new data
    df_old_cluster, df_new_cluster = cleansing_function(df_old_cluster, 
                                                        df_new_cluster, 
                                                        province, district, 
                                                        convert_phonenumber)
    
    
    print('Number of rows in df_old: {:,}'.format(df_old_cluster.shape[0]))
    print('Number of rows in df_new: {:,}'.format(df_new_cluster.shape[0]))
    print('Expected number of rows after unification: {:,}'.format(df_new_cluster.shape[0] + df_old_cluster.shape[0]))
    
    # Unification
    df_unified_cluster = unification_old_new_function(df_old_cluster, 
                                                      df_new_cluster, 
                                                      convert_phonenumber)
    
    df_unified_cluster['key'] = df_unified_cluster.apply(lambda x: generate_key(x['vin_cleansing'], 
                                                                  x['name_cleansing'], 
                                                                  x['address_cleansing'], 
                                                                  x['phone_cleansing'], 
                                                                  x['district_group'], 
                                                                  x['city_group']), axis=1)
    
    df_unified_cluster = pd.merge(left = df_combi_cluster, right = df_unified_cluster, how='left', on='key')
    df_unified_cluster = df_unified_cluster[['unification_id_y', 'customer_code', 'vin', 'customer_name', 
                                           'customer_address', 'mobile_phone', 
                                           'name_cleansing', 'address_cleansing', 'phone_cleansing', 
                                           'district', 'city', 'cluster']]
    df_unified_cluster = df_unified_cluster.rename(columns = {'unification_id_y' : 'unification_id'})
    df_unified_cluster.to_csv(output_name, index=False)
    print('Actual number of rows after unification: {:,}'.format(df_unified_cluster.shape[0]))
    print('Output saved:', output_name)

Unifying case "match_vin_only" cluster "0"
Number of rows in df_old: 92,209
Number of rows in df_new: 69,650
Expected number of rows after unification: 161,859


Processing rows:  77%|███████▋  | 53365/69650 [3:38:13<1:12:15,  3.76it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [9]:
df_unified_cluster

,unification_id,customer_code,vin,customer_name,customer_address,mobile_phone,name_cleansing,address_cleansing,phone_cleansing,district,city,cluster
0,0e5d2910531d6bab3c73c9cacfbcbf85d4653cc7c3070e...,MH_4152397,000002883K5133876,NGO TUAN DUNG,LONG BIÊN HÀ NỘI,0904382002,ngotuandung,,0904382002,longbien,hanoi,0
1,94b327f43e3666ef5f167105bddf659f4f3c1cd17f6dc8...,MH_3401434,000004322AY237320,HUỲNH VĂN MẪN,42 6 HOÀNG BẬT ĐẠT P 15 TÂY HOÀ HỒ CHÍ MINH,0399911084,huynhvanman,426hoangbatdat15tayhoa,0399911084,other,hochiminh,0
2,953378e6739b8afe1055171648fc68805c5e805e3e48db...,MH_225318,00000C100M0192976,BẠCH QUỐC VIỆT,SỐ 21 LK25 KĐT 5A PHƯỜNG 4 SÓC TRĂNG,0939952777,bachquocviet,21lk25kdt5a4,0939952777,other,soctrang,0
3,1273a44fd422bcef0ec306ca6ca89bbf0430c480d94979...,MH_425091,00000KC26E1043408,TRẦN VĂN THÌN,TRƯỜNG KHÁNH LỘC NINH SÓC TRĂNG,0868089841,tranvanthin,truongkhanhlocninh,0868089841,other,soctrang,0
4,953378e6739b8afe1055171648fc68805c5e805e3e48db...,MH_225318,1010KY137909EXCIT,BẠCH QUỐC VIỆT,SỐ 21 LK25 KĐT 5A PHƯỜNG 4 SÓC TRĂNG,0939952777,bachquocviet,21lk25kdt5a4,0939952777,other,soctrang,0
...,...,...,...,...,...,...,...,...,...,...,...,...
162680,3a022c5873a7cccaf2f7331894b24eacdfb101b9826822...,MH_1068484,WAVE5266DY0220402,NGUYỄN THANH BÌNH,THÔN 2 XÃ EAĐAR HUYỆN EAKAR TỈNH ĐĂKLAK EA KAR...,0365834613,nguyenthanhbinh,2eadar,0365834613,eakar,daklak,0
162681,a2f4bc0f623d3ab58889c9e2f497c218c104440569b40c...,MH_469058,Y053745JC76E10769,LÊ THỊ ĐẬM,HỒ HỌC LÃM KHÁNH VĨNH HỒ CHÍ MINH,0907198818,lethidam,hohoclamkhanhvinh,0907198818,other,hochiminh,0
162682,1974d0555494f000377fb8d11d52a8fc9734af54a82058...,MH_9012292,ZDCKF10B0AF059950,TRẦN THANH HẬN,PHƯỚC LÝ CẦN GIUỘC LONG AN PHAN THIẾT LONG AN,0982957990,tranthanhhan,phuoclycangiuocphanthiet,0982957990,other,longan,0
162683,876ccf6afc414404da683fa51ddc13e621650d6b12faa9...,MH_2622484,ZDCKF10BOAF056343,NGUYEN VAN DO,HỒ CHÍ MINH,0933383338,nguyenvando,,0933383338,other,hochiminh,0


In [12]:
# df_unified_cluster_copy = df_unified_cluster.copy()
# df_unified_cluster_copy = df_unified_cluster_copy[['unification_id_y', 'customer_code', 'vin', 'customer_name', 
#                                            'customer_address', 'mobile_phone', 
#                                            'name_cleansing', 'address_cleansing', 'phone_cleansing', 
#                                            'district', 'city', 'cluster']]
# df_unified_cluster_copy = df_unified_cluster_copy.rename(columns = {'unification_id_y' : 'unification_id'})
# df_unified_cluster_copy.to_csv(output_name, index=False)

In [13]:
# df_unified_cluster_copy